In [ ]:
"""from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from xgboost import XGBClassifier

# Dropping columns
X = df.drop(["DIAS INSTITUICAO LABEL", "DIAS NA INSTITUICAO", "DIAS NA INSTITUICAO_CATEGORIA"], axis=1)
y = df["DIAS INSTITUICAO LABEL"]

# Creating an imputer that replaces NaN with the column's median
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.25, random_state=42)

# Applying SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
X_test_resampled, y_test_resampled = smote.fit_resample(X_test, y_test)

# Normalizing the data
scaler = MinMaxScaler()
X_train_resampled_normalized = scaler.fit_transform(X_train_resampled)
X_test_resampled_normalized = scaler.transform(X_test_resampled)

# Define the pipeline with SelectFromModel and RandomForestClassifier for feature selection
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(estimator=RandomForestClassifier(random_state=42))),
    ('classification', XGBClassifier())
])

# Define the grid of parameters to search
param_grid = {
    'feature_selection__estimator__n_estimators': [10, 30, 50, 70, 100],
    'classification__n_estimators': [50, 100, 130],
    'classification__learning_rate': [0.01, 0.1, 0.2],
    'classification__max_depth': [3, 5, 10],
    'classification__min_child_weight': [1, 2, 4]
}

# Configure the GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Train the GridSearchCV
grid_search.fit(X_train_resampled_normalized, y_train_resampled)

# Get the best parameters and best model
best_params = grid_search.best_params_
print("Melhores parâmetros encontrados: ", best_params)
best_model = grid_search.best_estimator_

# Get the selected features using the best model
rfe = best_model.named_steps['feature_selection']
selected_features_indices = np.where(rfe.get_support())[0]

# Use the selected features for the XGBClassifier
X_train_selected = X_train_resampled_normalized[:, selected_features_indices]
X_test_selected = X_test_resampled_normalized[:, selected_features_indices]

# Train the GradientBoostingClassifier model with the selected features
classifier = best_model.named_steps['classification']
classifier.fit(X_train_selected, y_train_resampled)
predictions = classifier.predict(X_test_selected)
probabilities = classifier.predict_proba(X_test_selected)

# Calculate and display the AUC-ROC score for multi-class
auc_roc = roc_auc_score(y_test_resampled, probabilities, multi_class='ovr')
print("AUC-ROC Score:", auc_roc)

# Displaying the results
print("Feature Selector: SelectFromModel with RandomForest, Classifier: XGBClassifier")
print(classification_report(y_test_resampled, predictions))
print(confusion_matrix(y_test_resampled, predictions))

# Performing k-fold cross-validation on the training set only
Xjoined_array = np.concatenate((X_train_selected, X_test_selected), axis=0)
yjoined_array = np.concatenate((y_train_resampled, y_test_resampled), axis=0)
#cv_scores = cross_val_score(classifier, Xjoined_array, yjoined_array, cv=5, scoring='roc_auc_ovr')

# Displaying cross-validation results
#print("Cross-Validation Scores:", cv_scores)
#print("Mean CV Score:", np.mean(cv_scores))
"""